### 1. 데이터 API
- 일반적으로 디스크에서 데이터를 점진적으로 읽는 데이터셋 사용
- 간단히 tf.data.Dataset.from_tensor_slices()를 사용해 메모리에서 데이터셋 생성해보기

In [69]:
import tensorflow as tf
from tensorflow import keras

In [15]:
# tf.data.Dataset.from_tensor_slices()
# 각 원소가 아이템으로 표현되는 tf.data.Dataset 만들기

X = tf.range(10)
dataset = tf.data.Dataset.from_tensor_slices(X) # tf.data.Dataset.range(10) 동일
dataset

<TensorSliceDataset shapes: (), types: tf.int32>

In [5]:
for item in dataset:   # 아이템 순회가능
    print(item)

tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(5, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(7, shape=(), dtype=int32)
tf.Tensor(8, shape=(), dtype=int32)
tf.Tensor(9, shape=(), dtype=int32)


#### 1) 연쇄 변환

In [16]:
# 3번 반복하고 7개씩 묶어서 item으로 만들기

dataset = dataset.repeat(3).batch(7)  # drop_reminder=True 하면 길이부족 배치 버림

for item in dataset:
    print(item)

tf.Tensor([0 1 2 3 4 5 6], shape=(7,), dtype=int32)
tf.Tensor([7 8 9 0 1 2 3], shape=(7,), dtype=int32)
tf.Tensor([4 5 6 7 8 9 0], shape=(7,), dtype=int32)
tf.Tensor([1 2 3 4 5 6 7], shape=(7,), dtype=int32)
tf.Tensor([8 9], shape=(2,), dtype=int32)


In [17]:
# 다양한 변환

dataset = dataset.map(lambda x: x * 2)

In [18]:
dataset = dataset.apply(tf.data.experimental.unbatch())

In [21]:
dataset = dataset.filter(lambda x: x < 10)

In [22]:
for item in dataset:
    print(item)

tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(8, shape=(), dtype=int32)
tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(8, shape=(), dtype=int32)
tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(8, shape=(), dtype=int32)


#### 2) 데이터 셔플링

In [6]:
dataset = tf.data.Dataset.range(10).repeat(3)  # 3번 반복
dataset = dataset.shuffle(buffer_size=5, seed=42).batch(7)
   # 반복마다 동일한 순서사용 -> reshuffle_each_iteration=False
for item in dataset:
    print(item)

tf.Tensor([0 2 3 6 7 9 4], shape=(7,), dtype=int64)
tf.Tensor([5 0 1 1 8 6 5], shape=(7,), dtype=int64)
tf.Tensor([4 8 7 1 2 3 0], shape=(7,), dtype=int64)
tf.Tensor([5 4 2 7 8 9 9], shape=(7,), dtype=int64)
tf.Tensor([3 6], shape=(2,), dtype=int64)


##### 여러 파일에서 한 줄씩 번갈아 읽기

In [29]:
# 데이터 준비

# import pandas as pd
# import numpy as np
# import os
# from sklearn.datasets import fetch_california_housing
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import StandardScaler

# housing = fetch_california_housing()
# X_train_full, X_test, y_train_full, y_test = train_test_split(
#     housing.data, housing.target.reshape(-1, 1), random_state=42)
# X_train, X_valid, y_train, y_valid = train_test_split(
#     X_train_full, y_train_full, random_state=42)

# scaler = StandardScaler()
# scaler.fit(X_train)
# X_mean = scaler.mean_
# X_std = scaler.scale_

# def save_to_multiple_csv_files(data, name_prefix, header=None, n_parts=10):
#     housing_dir = os.path.join("datasets", "housing")
#     os.makedirs(housing_dir, exist_ok=True)
#     path_format = os.path.join(housing_dir, "my_{}_{:02d}.csv")

#     filepaths = []
#     m = len(data)
#     for file_idx, row_indices in enumerate(np.array_split(np.arange(m), n_parts)):
#         part_csv = path_format.format(name_prefix, file_idx)
#         filepaths.append(part_csv)
#         with open(part_csv, "wt", encoding="utf-8") as f:
#             if header is not None:
#                 f.write(header)
#                 f.write("\n")
#             for row_idx in row_indices:
#                 f.write(",".join([repr(col) for col in data[row_idx]]))
#                 f.write("\n")
#     return filepaths


# train_data = np.c_[X_train, y_train]
# valid_data = np.c_[X_valid, y_valid]
# test_data = np.c_[X_test, y_test]
# header_cols = housing.feature_names + ["MedianHouseValue"]
# header = ",".join(header_cols)

# train_filepaths = save_to_multiple_csv_files(train_data, "train", header, n_parts=20)
# valid_filepaths = save_to_multiple_csv_files(valid_data, "valid", header, n_parts=10)
# test_filepaths = save_to_multiple_csv_files(test_data, "test", header, n_parts=10)

In [31]:
train_filepaths  # train_filepaths = "datasets/housing/my_train_*.csv"로 쓸 수 있음

['datasets\\housing\\my_train_00.csv',
 'datasets\\housing\\my_train_01.csv',
 'datasets\\housing\\my_train_02.csv',
 'datasets\\housing\\my_train_03.csv',
 'datasets\\housing\\my_train_04.csv',
 'datasets\\housing\\my_train_05.csv',
 'datasets\\housing\\my_train_06.csv',
 'datasets\\housing\\my_train_07.csv',
 'datasets\\housing\\my_train_08.csv',
 'datasets\\housing\\my_train_09.csv',
 'datasets\\housing\\my_train_10.csv',
 'datasets\\housing\\my_train_11.csv',
 'datasets\\housing\\my_train_12.csv',
 'datasets\\housing\\my_train_13.csv',
 'datasets\\housing\\my_train_14.csv',
 'datasets\\housing\\my_train_15.csv',
 'datasets\\housing\\my_train_16.csv',
 'datasets\\housing\\my_train_17.csv',
 'datasets\\housing\\my_train_18.csv',
 'datasets\\housing\\my_train_19.csv']

In [32]:
# list_files() : 파일 경로를 섞은 데이터 셋 반환
# 섞지 않으려면 shuffle=False 옵션 사용

filepath_dataset = tf.data.Dataset.list_files(train_filepaths, seed=42)

In [34]:
for item in filepath_dataset:
    print(item)

tf.Tensor(b'datasets\\housing\\my_train_05.csv', shape=(), dtype=string)
tf.Tensor(b'datasets\\housing\\my_train_16.csv', shape=(), dtype=string)
tf.Tensor(b'datasets\\housing\\my_train_01.csv', shape=(), dtype=string)
tf.Tensor(b'datasets\\housing\\my_train_17.csv', shape=(), dtype=string)
tf.Tensor(b'datasets\\housing\\my_train_00.csv', shape=(), dtype=string)
tf.Tensor(b'datasets\\housing\\my_train_14.csv', shape=(), dtype=string)
tf.Tensor(b'datasets\\housing\\my_train_10.csv', shape=(), dtype=string)
tf.Tensor(b'datasets\\housing\\my_train_02.csv', shape=(), dtype=string)
tf.Tensor(b'datasets\\housing\\my_train_12.csv', shape=(), dtype=string)
tf.Tensor(b'datasets\\housing\\my_train_19.csv', shape=(), dtype=string)
tf.Tensor(b'datasets\\housing\\my_train_07.csv', shape=(), dtype=string)
tf.Tensor(b'datasets\\housing\\my_train_09.csv', shape=(), dtype=string)
tf.Tensor(b'datasets\\housing\\my_train_13.csv', shape=(), dtype=string)
tf.Tensor(b'datasets\\housing\\my_train_15.csv', sh

In [35]:
# interleave() : 한 번에 다섯 개의 파일 한 줄씩 번갈아 읽기
# 병렬화를 사용하진 않지만, num_parallel_calls로 병렬화 가능

n_readers = 5
dataset = filepath_dataset.interleave(
    lambda filepath: tf.data.TextLineDataset(filepath).skip(1),
    cycle_length=n_readers)   # 각 파일의 첫 줄은 열이름이므로 skip

In [44]:
# 데이터 살펴보기 
# 순서는 랜덤이며, 바이트 스트링으로 되어 있음 (파싱하고 스케일 조정해야)

for line in dataset.take(5):
    print(line.numpy())

b'2.1856,41.0,3.7189873417721517,1.0658227848101265,803.0,2.0329113924050635,32.76,-117.12,1.205'
b'4.2708,45.0,5.121387283236994,0.953757225433526,492.0,2.8439306358381504,37.48,-122.19,2.67'
b'3.8456,35.0,5.461346633416459,0.9576059850374065,1154.0,2.8778054862842892,37.96,-122.05,1.598'
b'3.0217,22.0,4.983870967741935,1.1008064516129032,615.0,2.4798387096774195,38.76,-120.6,1.069'
b'4.2083,44.0,5.323204419889502,0.9171270718232044,846.0,2.3370165745856353,37.47,-122.2,2.782'


#### 3) 데이터 전처리

In [48]:
X_mean, X_std   # 훈련셋의 각 특성의 평균과 표준편차

(array([ 3.89175860e+00,  2.86245478e+01,  5.45593655e+00,  1.09963474e+00,
         1.42428122e+03,  2.95886657e+00,  3.56464315e+01, -1.19584363e+02]),
 array([1.90927329e+00, 1.26409177e+01, 2.55038070e+00, 4.65460128e-01,
        1.09576000e+03, 2.36138048e+00, 2.13456672e+00, 2.00093304e+00]))

In [53]:
n_inputs = 8

def preprocess(line):
    defs = [0.] * n_inputs + [tf.constant([], dtype=tf.float32)]
       # defs : [0, 0, 0, ..., 0, tf.Tensor]  -- X부분은 0으로 y는 텐서로
    fields = tf.io.decode_csv(line, record_defaults=defs)
       # record_defaults는 기본값 배열(누락값의 기본값을 0으로, y는 기본값 X)
    
    x = tf.stack(fields[:-1])  # stack은 1D 텐서 배열을 반환
    y = tf.stack(fields[-1:])
    return (x - X_mean) / X_std, y

In [54]:
preprocess(b'2.1856,41.0,3.7189873417721517,1.0658227848101265,803.0,2.0329113924050635,32.76,-117.12,1.205')

(<tf.Tensor: shape=(8,), dtype=float32, numpy=
 array([-0.8936168 ,  0.9789995 , -0.6810549 , -0.07264194, -0.5669866 ,
        -0.39212456, -1.3522334 ,  1.2316071 ], dtype=float32)>,
 <tf.Tensor: shape=(1,), dtype=float32, numpy=array([1.205], dtype=float32)>)

#### 4) 데이터 적재와 전처리 합치기
<img src='img/13_1.png' width='500'>

In [59]:
# 재사용가능한코드를 만들기 위해 지금까지의 코드를 하나의 헬퍼 함수로 만들기
# 적재, 전처리, 셔플링, 반복, 배치를 적용한 데이터 반환

def csv_reader_dataset(filepaths, repeat=1, n_readers=5,
                      n_read_threads=None, shuffle_buffer_size=10000,
                      n_parse_threads=5, batch_size=32):
    
    dataset = tf.data.Dataset.list_files(filepaths).repeat(repeat)
    dataset = dataset.interleave(
        lambda filepath: tf.data.TextLineDataset(filepath).skip(1),
        cycle_length=n_readers, num_parallel_calls=n_read_threads)
    dataset = dataset.shuffle(shuffle_buffer_size)
    dataset = dataset.map(preprocess, num_parallel_calls=n_parse_threads)
    return dataset.batch(batch_size).prefetch(1)  # 아래에서 살펴봄

#### 5) 프리패치
- 마지막에 prefetch(1)를 호출하면 데이터셋은 항상 한 배치가 미리 준비되도록!  
즉, 한 배치를 훈련하는 동안 다음 배치를 준비함

#### 6) tf.keras와 데이터셋 사용하기

In [60]:
train_set = csv_reader_dataset(train_filepaths)
valid_set = csv_reader_dataset(valid_filepaths)
test_set = csv_reader_dataset(test_filepaths)

In [63]:
# 바로 사용가능 

# model.fit(train_set, epochs=10, validation_data=valid_set)
# model . evaluate (test_set)

In [64]:
# 전체 훈련 반복 수행하는 텐서플로 함수 직접 만들수도 있음

# @tf.function
# def train(model, optimizer, loss_fn, n_epochs, [...]):
#     train_set = csv_reader_dataset(train_filepaths, repeat=n_epochs, [...])
#     for X_batch, y_batch in train_set:
#         with tf.GradientTape() as tape:
#             y_pred = model(X_batch)
#             main_loss = tf.reduce_mean(loss_fn(y_batch, y_pred))
#             loss = tf.add_n([main_loss] + model.losses)
#         grads = tape.gradient(loss, model.trainable_variables)
#         optimizer.apply_gradient(zip(grads, model.trainable_variables))

### 2. TFRecord 포멧
- csv 파일이 간단하고 편리해서 널리 통용되지만 대규모의 복잡한 (이미지나 오디오 등) 데이터는 지원 X => TFRecord 포맷
- TFRecord : 크기가 다른 연속된 이진 레코드를 저장하는 단순한 이진 포맷

In [65]:
with tf.io.TFRecordWriter('datasets/my_data.tfrecord') as f:
    f.write(b"This is the first record")
    f.write(b"And this is the second record")

In [66]:
filepaths = ['datasets/my_data.tfrecord']
dataset = tf.data.TFRecordDataset(filepaths)
for item in dataset:
    print(item)

tf.Tensor(b'This is the first record', shape=(), dtype=string)
tf.Tensor(b'And this is the second record', shape=(), dtype=string)


- (이하 생략) TFRecord 포맷이 필요할 때, 다시 공부해보기

### 3. 입력 특성 전처리
- 모든 특성을 수치 특성으로 변환하고 정규화해야!
- 도구를 이용해서 하거나, map 메서드를 이용하여 동적으로 처리하거나 전처리 층을 모델시킬 수 있음

##### Lambda 층을 이용하기

In [ ]:
means = np.mean(X_train, axis=0, keepdims=True)
stds = np.std(X_train, axis=0, keepdims=True)
eps = keras.backend.epsilon()
model = keras.models.Sequential([
    keras.layers.Lambda(lambda inputs: (inputs - means) / (stds + eps)),
    [...]
])

##### (means와 같은 전역 변수를 다루기보다) 사용자 정의 층 만들 수 있음

In [ ]:
class Standardization(keras.layers.Layer):
    def adapt(self, data_sample):
        self.means_ = np.mean(data_sample, axis=0, keepdims=True)
        self.stds_ = np.std(data_sample, axis=0, keepdims=True)
        
    def call(self, inputs):
        return (inputs - self.means_) / (self.stds_ + keras.backend.epsilon())
    

In [70]:
std_layer = Standardization()
std_layer.adapt(data_sample)  # 이때, data_sample이 전체일 필요 X 

model = keras.Sequential()
model.add(std_layer)
[...]

- keras.layers.Normalization 층 사용가능

#### 1) 원-핫 벡터 사용하여 범주형 특성 인코딩하기

In [94]:
vocab = [ "<1H OCEAN" , "INLAND" , "NEAR OCEAN" , "NEAR BAY", "ISLAND" ]
indices = tf.range(len(vocab), dtype=tf.int64)

# tf.lookup.KeyValueTensorInitializer : 범주 리스트와 해당 인덱스를를 전달하여
                                    # 룩업 테이블을 위해 초기화 객체를 만듦
table_init = tf.lookup.KeyValueTensorInitializer(vocab, indices)


# 어휘 사전에 없는 번주를 찾으면 룩업 테이블이 계산한 이 범주의 해시값을 이용해
# oov(out-of-vocabulary) 버킷 중 하나에 할당 (있는 범주 다음부터! 여기서는 5, 6)
num_oov_buckets = 2
table = tf.lookup.StaticVocabularyTable(table_init, num_oov_buckets)

- oov 버킷을 사용하는 이유  
: 범주의 개수가 많은 경우, 전체 범주 리스트를 구하는 것이 어려움  
=> 샘플 데이터를 기반으로 어휘사전을 정의하고 데이터에 없는 범주를 oov 버킷에 추가 (oov 버킷이 충분해야!)

In [81]:
# 룩업 테이블 사용해 원-핫 벡터로 인코딩해보기

categories = tf.constant(['NEAR BAY', 'DESERT', 'INLAND', 'INLAND'])
cat_indices =  table.lookup(categories)
cat_indices

<tf.Tensor: shape=(4,), dtype=int64, numpy=array([3, 5, 1, 1], dtype=int64)>

In [83]:
cat_one_hot = tf.one_hot(cat_indices, depth=len(vocab) + num_oov_buckets)
cat_one_hot

<tf.Tensor: shape=(4, 7), dtype=float32, numpy=
array([[0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0.]], dtype=float32)>

- keras.layers.TextVectorization 층 사용가능
- 어휘 사전이 크면 __임베딩__을 사용하여 인코딩하는 것이 훨씬 효율적

#### 2) 임베딩을 사용해 범주형 특성 인코딩하기
- __임베딩__ : 범주를 표현하는 훈련 가능한 밀집 벡터 
- 임베딩을 훈련할 수 있음 -> 비슷한 범주들 가깝게 만듦
- 표현이 좋을수록 신경망이 정확한 예측을 만듦 (표현학습(17장 참고))

   cf) 단어임베딩
- 일반적으로 직접 단어임베딩을 훈련하는 것보다 사전훈련된 임베딩을 재사용하는 게 나음
- King-Man, Queen-Woman과 같은 의미를 조직할 수 있음

##### 임베딩 직접 구현해보기 (동작원리를 이용하기 위해) 

In [84]:
embedding_dim = 2
embed_init = tf.random.uniform([len(vocab) + num_oov_buckets, embedding_dim])
embed_init

<tf.Tensor: shape=(7, 2), dtype=float32, numpy=
array([[0.09632993, 0.9395069 ],
       [0.09627533, 0.7377459 ],
       [0.5443243 , 0.4047718 ],
       [0.7686013 , 0.8976238 ],
       [0.18245184, 0.8822814 ],
       [0.50123084, 0.02763808],
       [0.65395284, 0.3409189 ]], dtype=float32)>

In [86]:
embedding_matrix = tf.Variable(embed_init)
embedding_matrix

# 변수에 저장되며, 훈련과정에서 경사하강법으로 학습가능

<tf.Variable 'Variable:0' shape=(7, 2) dtype=float32, numpy=
array([[0.09632993, 0.9395069 ],
       [0.09627533, 0.7377459 ],
       [0.5443243 , 0.4047718 ],
       [0.7686013 , 0.8976238 ],
       [0.18245184, 0.8822814 ],
       [0.50123084, 0.02763808],
       [0.65395284, 0.3409189 ]], dtype=float32)>

In [88]:
# 위의 예제 인코딩해보기

categories = tf.constant(['NEAR BAY', 'DESERT', 'INLAND', 'INLAND'])
cat_indices = table.lookup(categories)
cat_indices

<tf.Tensor: shape=(4,), dtype=int64, numpy=array([3, 5, 1, 1], dtype=int64)>

In [89]:
tf.nn.embedding_lookup(embedding_matrix, cat_indices)  # [3, 5, 1, 1]해당 벡터

<tf.Tensor: shape=(4, 2), dtype=float32, numpy=
array([[0.7686013 , 0.8976238 ],
       [0.50123084, 0.02763808],
       [0.09627533, 0.7377459 ],
       [0.09627533, 0.7377459 ]], dtype=float32)>

##### 케라스에서 임베딩 행렬을 처리해주는 keras.layers.Embedding 층

In [95]:
embedding = keras.layers.Embedding(input_dim=len(vocab) + num_oov_buckets,
                                  output_dim=embedding_dim)

In [96]:
embedding(cat_indices)

<tf.Tensor 'embedding_2/Identity:0' shape=(None, 2) dtype=float32>

In [90]:
# 모두를 연결해보기 -> 임베딩을 학습하는 케라스 모델 만들 수 있음

regular_inputs = keras.layers.Input(shape=[8])
categories = keras.layers.Input(shape=[], dtype=tf.string)
cat_indices = keras.layers.Lambda(lambda cats: table.lookup(cats))(categories)
cat_embed = keras.layers.Embedding(input_dim=6, output_dim=2)(cat_indices)
encoded_inputs = keras.layers.concatenate([regular_inputs, cat_embed])
outputs = keras.layers.Dense(1)(encoded_inputs)  # Embedding 층과 동일한 역할
model = keras.models.Model(inputs=[regular_inputs, categories],
                          outputs=[outputs])

# 이 모델은 8개의 특성을 담은 입력과 하나의 범주형 입력을 가짐
# Lambda 층을 사용해 범주의 인덱스를 찾은 다음 임베딩에서 이 인덱스를 찾음
# 그다음 이 임베딩과 입반 입력을 연결

#### 3) 케라스 전처리 층

In [ ]:
normalization = keras.layers.Normalization()
discretization = keras.layers.Discretization([...]) 
     # 연속적인 데이터를 구간으로 나누고 원-핫 벡터로 인코딩
pipeline = keras.layers.PreprocessingStage([normalization, discretization])
     # 여러개의 전처리층 합치기 (정규화하고 이산화하기)
pipeline.adapt(data_sample)

- TextVectorization 층 : 단어 카운트 벡터를 출력  
 ex) 어휘사전이 ['and', 'basketball', 'more']이라면 'more and more' 텍스트는 벡터 [1, 0, 2]로 매핑됨 (단어의 개수)  
 => 이런 텍스트 표현은 순서를 완전히 무시하기 때문에 __BOW__라 부름
 
    cf) TF-IDF   
    = tf * log(1 + ${전체 샘플 수} \over {1+단어가 등장하는 샘플 수}$)

### 4. TF 변환
- 전처리 연산을 한 번만 정의하도록 하는 TF 변환
- 텐서플로 모델 상품화의 엔드-투-엔드(end-to-end) 플랫폼 TFX의 일부분

### 5. 텐서플로 데이터셋(TFDS)

In [98]:
import tensorflow_datasets as tfds

In [106]:
dataset = tfds.load(name='mnist')
mnist_train, mnist_test = dataset['train'], dataset['test']

In [108]:
mnist_train = mnist_train.shuffle(10000).batch(32)
mnist_train = mnist_train.map(lambda items: (items['image'], items['label']))
                                           # 튜플로 만들어주기
mnist_train = mnist_train.prefetch(1)

In [ ]:
# 더 간단히 (as_supervies=True 옵션 사용)

dataset = tfds.load(name='mnist', batch_size=32, as_supervised=True)
mnist_train = dataset['train'].prefetch(1)

[...]
model.fit(mnist_train, epochs=5)